# Combining Datasets: Concat and Append

Some of the most interesting studies of data come from combining different data sources.
These operations can involve anything from very straightforward concatenation of two different datasets, to more complicated database-style joins and merges that correctly handle any overlaps between the datasets.
``Series`` and ``DataFrame``s are built with this type of operation in mind, and Pandas includes functions and methods that make this sort of data wrangling fast and straightforward.

Here we'll take a look at simple concatenation of ``Series`` and ``DataFrame``s with the ``pd.concat`` function; later we'll dive into more sophisticated in-memory merges and joins implemented in Pandas.

We begin with the standard imports:

In [ ]:
import pandas as pd
import numpy as np

## Recall: Concatenation of NumPy Arrays

Concatenation of ``Series`` and ``DataFrame`` objects is very similar to concatenation of Numpy arrays, which can be done via the ``np.concatenate`` function.

In [ ]:
x = [1, 2, 3]
y = [4, 5, 6]
z = [7, 8, 9]
np.concatenate([x, y, z])

In [ ]:
np.concatenate([x, y, z],axis=0)

In [ ]:
np.concatenate([x, y, z],axis=1)

The first argument is a list or tuple of arrays to concatenate.
Additionally, it takes an ``axis`` keyword that allows you to specify the axis along which the result will be concatenated:

In [ ]:
x = [[1, 2],
     [3, 4]]
x = [[1, 2],
     [3, 4]]
np.concatenate([x, x], axis=0) # add more row

It specifies the ``axis`` along which the means are computed. By default axis=0. This is consistent with the numpy.mean usage when axis is specified explicitly (in numpy.mean, axis==None by default, which computes the mean value over the flattened array) , in which axis=0 along the rows (namely, index in pandas), and axis=1 along the columns. For added clarity, one may choose to specify axis='index' (instead of axis=0) or axis='columns' (instead of axis=1).

In [ ]:
x = [[1, 2],
     [3, 4]]
np.concatenate([x, x], axis=1) # add more columns

## Simple Concatenation with ``pd.concat``

Pandas has a function, ``pd.concat()``, which has a similar syntax to ``np.concatenate`` but contains a number of options that we'll discuss momentarily:

```python
# Signature in Pandas v0.18
pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=True)
```

``pd.concat()`` can be used for a simple concatenation of ``Series`` or ``DataFrame`` objects, just as ``np.concatenate()`` can be used for simple concatenations of arrays:

In [ ]:
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])


In [ ]:
ser1

In [ ]:
ser2

In [ ]:
pd.concat([ser1, ser2])

It also works to concatenate higher-dimensional objects, such as ``DataFrame``s:

In [ ]:
def make_df(cols, ind):
    """Quickly make a DataFrame"""
    data = {c: [str(c) + str(i) for i in ind]
            for c in cols}
    return pd.DataFrame(data, ind)

# example DataFrame
make_df('ABC', range(3))

In [ ]:
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [3, 4])
df1

In [ ]:
df2

In [ ]:
pd.concat([df1, df2])

By default, the concatenation takes place row-wise within the ``DataFrame`` (i.e., ``axis=0``).
Like ``np.concatenate``, ``pd.concat`` allows specification of an axis along which concatenation will take place.
Consider the following example:

In [ ]:
df3 = make_df('AB', [0, 1])
df4 = make_df('CD', [0, 1])
df3

In [ ]:
df4

In [ ]:
pd.concat([df3, df4], axis=1)

We could have equivalently specified ``axis=1``; here we've used the more intuitive ``axis='col'``. 

### Duplicate indices

One important difference between ``np.concatenate`` and ``pd.concat`` is that Pandas concatenation *preserves indices*, even if the result will have duplicate indices!
Consider this simple example:

In [ ]:
x = make_df('AB', [0, 1])
y = make_df('AB', [2, 3])
y.index = x.index  # make duplicate indices!
x

In [ ]:
y

In [ ]:
pd.concat([x, y],axis='rows')

In [ ]:
pd.concat([x, y],axis=1)

# Combining Datasets: Merge and Join

One essential feature offered by Pandas is its high-performance, in-memory join and merge operations.
If you have ever worked with databases, you should be familiar with this type of data interaction.
The main interface for this is the ``pd.merge`` function, and we'll see few examples of how this can work in practice.


## Relational Algebra

The behavior implemented in ``pd.merge()`` is a subset of what is known as *relational algebra*, which is a formal set of rules for manipulating relational data, and forms the conceptual foundation of operations available in most databases.
The strength of the relational algebra approach is that it proposes several primitive operations, which become the building blocks of more complicated operations on any dataset.
With this lexicon of fundamental operations implemented efficiently in a database or other program, a wide range of fairly complicated composite operations can be performed.

Pandas implements several of these fundamental building-blocks in the ``pd.merge()`` function and the related ``join()`` method of ``Series`` and ``Dataframe``s.
As we will see, these let you efficiently link data from different sources.

## Categories of Joins

The ``pd.merge()`` function implements a number of types of joins: the *one-to-one*, *many-to-one*, and *many-to-many* joins.
All three types of joins are accessed via an identical call to the ``pd.merge()`` interface; the type of join performed depends on the form of the input data.
Here we will show simple examples of the three types of merges, and discuss detailed options further below.

### One-to-one joins

Perhaps the simplest type of merge expresion is the one-to-one join, which is in many ways very similar to the column-wise concatenation
As a concrete example, consider the following two ``DataFrames`` which contain information on several employees in a company:

In [ ]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
df1

In [ ]:
df2

To combine this information into a single ``DataFrame``, we can use the ``pd.merge()`` function:

In [ ]:
df3 = pd.merge(df2, df1)
df3

The ``pd.merge()`` function recognizes that each ``DataFrame`` has an "employee" column, and automatically joins using this column as a key.
The result of the merge is a new ``DataFrame`` that combines the information from the two inputs.
Notice that the order of entries in each column is not necessarily maintained: in this case, the order of the "employee" column differs between ``df1`` and ``df2``, and the ``pd.merge()`` function correctly accounts for this.
Additionally, keep in mind that the merge in general discards the index, except in the special case of merges by index (see the ``left_index`` and ``right_index`` keywords, discussed momentarily).

### Many-to-one joins

Many-to-one joins are joins in which one of the two key columns contains duplicate entries.
For the many-to-one case, the resulting ``DataFrame`` will preserve those duplicate entries as appropriate.
Consider the following example of a many-to-one join:

In [ ]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
df4

In [ ]:
df3

In [ ]:
df6 = pd.merge(df3, df4)
df6

The resulting ``DataFrame`` has an aditional column with the "supervisor" information, where the information is repeated in one or more locations as required by the inputs.

### Many-to-many joins

Many-to-many joins are a bit confusing conceptually, but are nevertheless well defined.
If the key column in both the left and right array contains duplicates, then the result is a many-to-many merge.
This will be perhaps most clear with a concrete example.
Consider the following, where we have a ``DataFrame`` showing one or more skills associated with a particular group.
By performing a many-to-many join, we can recover the skills associated with any individual person:

In [ ]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})
df5

In [ ]:
df1

In [ ]:
pd.merge(df1, df5)

In [ ]:
pd.merge(df6, df5)

In [ ]:
pd.merge(df5, df6)

### Practice 

You are given two CSV files:

**students.csv**

| Student_ID | Name     | Major             |
|------------|----------|------------------|
| 1001       | Alice    | Computer Science |
| 1002       | Bob      | Math             |
| 1003       | Charlie  | Physics          |
| 1004       | Diana    | Economics        |

**scores.csv**

| Student_ID | Course    | Score |
|------------|-----------|-------|
| 1001       | CS101     | 95    |
| 1002       | MATH201   | 88    |
| 1003       | PHY111    | 72    |
| 1005       | BIO150    | 85    |

 🎯 **Tasks**

1. **Fill data** into pandas DataFrames.  
2. **Merge the DataFrames** on `Student_ID` using:
   - Inner join → keep only students present in both datasets.  
   - Left join → keep all students from `students.csv`, with matching scores where available.  
3. After each join, **print the resulting DataFrame** and explain the difference in the number of rows.  
4. Rename the `Score` column to `Final_Score` in the merged DataFrame.  
5. Save the final **outer join result** as `merged_results.csv`.

In [56]:
df = pd.DataFrame({'Student_ID': [1001, 1002, 1003, 1004],
                   'Name': ['Alice', 'Bob', 'Charlie', 'Diana'],
                   'Major': ['Computer Science', 'Math', 'Physics', 'Economics']})
df2 = pd.DataFrame({'Student_ID': [1001, 1002, 1003, 1005],
                   'Course': ['CS101', 'MATH201', 'PHY111', 'BIO150'],
                   'Score': [95, 88, 72, 85]})

In [57]:
inner_result = pd.merge(df, df2, on='Student_ID', how='inner')
left_result = pd.merge(df, df2, on='Student_ID', how='left')
inner_result

,Student_ID,Name,Major,Course,Score
0,1001,Alice,Computer Science,CS101,95
1,1002,Bob,Math,MATH201,88
2,1003,Charlie,Physics,PHY111,72


In [58]:
left_result


,Student_ID,Name,Major,Course,Score
0,1001,Alice,Computer Science,CS101,95.0
1,1002,Bob,Math,MATH201,88.0
2,1003,Charlie,Physics,PHY111,72.0
3,1004,Diana,Economics,NaN,NaN


**An inner merge** excludes the student with student_id 1004 because that student does not exist in both DataFrames. In contrast, a **left merge** includes all students from the left DataFrame, and for the student_id 1004 that has no match in the right DataFrame, it shows NaN values for the columns from the right side.
